<a href="https://colab.research.google.com/github/linzhi0918/deliberative-politics/blob/main/notebooks/Revised_01_Deliberation_Feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This codebook was run in Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""


@author: Kokil
"""


import pandas as pd
import csv
import numpy as np
import sys
import time
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.utils import class_weight
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.sparse import csr_matrix
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import sys
import os
import warnings
import pandas as pd
import json
import string
import re
import nltk
import random
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
#!pip install spacy
#!python3 -m spacy download en_core_web_lg
#!python3 -m spacy download en_core_web_sm

!pip install convokit
from convokit import Corpus, Speaker, Utterance
from convokit import download
from convokit import TextParser
from convokit import PolitenessStrategies
import spacy
import nltk
nltk.download('averaged_perceptron_tagger')
import sklearn.externals
import joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.1 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=2b075c42f9cdff255d39d63587c427e67db52612b535101cf20539b3e3b7eff4
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
nlp = spacy.load("en_core_web_sm")

def is_number(tok):
    try:
        float(tok)
        return True
    except ValueError:
        return False

def spacy_tokenizer(text):
    return [tok.text if not is_number(tok.text) else '_NUM_' for tok in nlp(text)]


In [ ]:
def extract_harbingers(df, X_col):

    with open('/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/2015_Diplomacy_lexicon.json') as f:
        features = json.loads(f.readline())

    for feature in features:
        harbingers = [harbinger.encode('ascii', 'ignore').decode('ascii').lower() for harbinger in features[feature]]
        features[feature] = harbingers

    def clean_text(text):
        text = str(text)
        text = text.replace('\'', '')
        text = text.lower()
        text = text.replace('{html}',"")
        text = re.sub(re.compile('<.*?>'), '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub('[0-9]+', '', text)
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(text)
        text = " ".join(tokens)
        return text

    def get_feature_frequency(text, feature):
        count = 0
        for harbinger in features[feature]:
            count += text.count(harbinger)
        return count

    df['clean_text'] = df.apply(lambda row: clean_text(row[X_col]), axis=1)
    for feature in features:
        df[feature] = df.apply(lambda row: get_feature_frequency(row['clean_text'], feature), axis=1)

In [ ]:
def preprocess(text_string):

        space_pattern = '\s+'
        giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|''[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        mention_regex = '@[\w\-]+'
        parsed_text = re.sub(space_pattern, ' ', text_string)
        parsed_text = re.sub(giant_url_regex, '', parsed_text)
        parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = parsed_text.code("utf-8", errors='ignore')
        return parsed_text


#nlp = en_core_web_sm.load()
#spacy.load("en_core_web_lg")
#spacy.load("en_core_web_sm")
ps = PolitenessStrategies()
spacy_nlp = spacy.load('en_core_web_sm', disable=['ner'])
cols = list(ps.transform_utterance("hello, could you please help me proofread this article?", spacy_nlp=spacy_nlp).meta['politeness_strategies'])

def extract_politeness_feats(df, X_col):

    def extract_politeness_helper(row):
        utt = ps.transform_utterance(row[X_col], spacy_nlp=spacy_nlp)
        feats = [utt.meta['politeness_strategies'][x] for x in cols]
        return pd.Series(feats)

    df[cols] = df.apply(extract_politeness_helper, axis=1)



Mounted at /content/drive


In [ ]:
# List harbingers, liwc and politeness features
import json
with open('/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/2015_Diplomacy_lexicon.json') as f:
    harb_dict = json.loads(f.readline())
politeness_dict = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/politeness_list.csv')
liwc_dict = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/liwc_list.csv')
X_cols = list(politeness_dict.columns) + list(liwc_dict.columns) + list(harb_dict.keys())

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
happierfuntokenizer_v3

This code implements a basic, Twitter-aware tokenizer.

A tokenizer is a function that splits a string of text into words. In
Python terms, we map string and unicode objects into lists of unicode
objects.

There is not a single right way to do tokenizing. The best method
depends on the application.  This tokenizer is designed to be flexible
and this easy to adapt to new domains and tasks.  The basic logic is
this:

1. The tuple regex_strings defines a list of regular expression
   strings.

2. The regex_strings strings are put, in order, into a compiled
   regular expression object called word_re.

3. The tokenization is done by word_re.findall(s), where s is the
   user-supplied string, inside the tokenize() method of the class
   Tokenizer.

4. When instantiating Tokenizer objects, there is a single option:
   preserve_case.  By default, it is set to True. If it is set to
   False, then the tokenizer will downcase everything except for
   emoticons.

The __main__ method illustrates by tokenizing a few examples.

I've also included a Tokenizer method tokenize_random_tweet(). If the
twitter library is installed (http://code.google.com/p/python-twitter/)
and Twitter is cooperating, then it should tokenize a random
English-language tweet.
"""

######################################################################

import re
import html.entities
######################################################################
# The following strings are components in the regular expression
# that is used for tokenizing. It's important that phone_number
# appears first in the final regex (since it can contain whitespace).
# It also could matter that tags comes after emoticons, due to the
# possibility of having text like
#
#     <:| and some text >:)
#
# Most imporatantly, the final element should always be last, since it
# does a last ditch whitespace-based tokenization of whatever is left.

# This particular element is used in a couple ways, so we define it
# with a name:
emoticon_string = r"""
    (?:
      [<>]?
      [:;=8>]                    # eyes
      [\-o\*\']?                 # optional nose
      [\)\]\(\[dDpPxX/\:\}\{@\|\\] # mouth
      |
      [\)\]\(\[dDpPxX/\:\}\{@\|\\] # mouth
      [\-o\*\']?                 # optional nose
      [:;=8<]                    # eyes
      [<>]?
      |
      <[/\\]?3                         # heart(added: has)
      |
      \(?\(?\#?                   #left cheeck
      [>\-\^\*\+o\~]              #left eye
      [\_\.\|oO\,]                #nose
      [<\-\^\*\+o\~]              #right eye
      [\#\;]?\)?\)?               #right cheek
    )"""

# The components of the tokenizer:
regex_strings = (
    # Phone numbers:
    r"""
    (?:
      (?:            # (international)
        \+?[01]
        [\-\s.]*
      )?
      (?:            # (area code)
        [\(]?
        \d{3}
        [\-\s.\)]*
      )?
      \d{3}          # exchange
      [\-\s.]*
      \d{4}          # base
    )"""
    ,
    # Emoticons:
    emoticon_string
    ,
    # http:
    # Web Address:
    r"""(?:(?:http[s]?\:\/\/)?(?:[\w\_\-]+\.)+(?:com|net|gov|edu|info|org|ly|be|gl|co|gs|pr|me|cc|us|gd|nl|ws|am|im|fm|kr|to|jp|sg)(?:\/[\s\b$])?)"""
    ,
    r"""(?:http[s]?\:\/\/)"""   #need to capture it alone sometimes
    ,
    #command in parens:
    r"""(?:\[[\w_]+\])"""   #need to capture it alone sometimes
    ,
    # HTTP GET Info
    r"""(?:\/\w+\?(?:\;?\w+\=\w+)+)"""
    ,
    # HTML tags:
    r"""(?:<[^>]+\w=[^>]+>|<[^>]+\s\/>|<[^>\s]+>?|<?[^<\s]+>)"""
    #r"""(?:<[^>]+\w+[^>]+>|<[^>\s]+>?|<?[^<\s]+>)"""
    ,
    # Twitter username:
    r"""(?:@[\w_]+)"""
    ,
    # Twitter hashtags:
    r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
    ,
    # Remaining word types:
    r"""
    (?:[\w][\w'\-_]+[\w])       # Words with apostrophes or dashes.
    |
    (?:[+\-]?\d+[,/.:-]\d+[+\-]?)  # Numbers, including fractions, decimals.
    |
    (?:[\w_]+)                     # Words without apostrophes or dashes.
    |
    (?:\.(?:\s*\.){1,})            # Ellipsis dots.
    |
    (?:\S)                         # Everything else that isn't whitespace.
    """
    )

######################################################################
# This is the core tokenizing regex:

word_re = re.compile(r"""(%s)""" % "|".join(regex_strings), re.VERBOSE | re.I | re.UNICODE)

# The emoticon string gets its own regex so that we can preserve case for them as needed:
emoticon_re = re.compile(regex_strings[1], re.VERBOSE | re.I | re.UNICODE)

# These are for regularizing HTML entities to Unicode:
html_entity_digit_re = re.compile(r"&#\d+;")
html_entity_alpha_re = re.compile(r"&\w+;")
amp = "&amp;"

hex_re = re.compile(r'\\x[0-9a-z]{1,4}')

######################################################################

class Tokenizer:
    def __init__(self, preserve_case=False, use_unicode=True):
        self.preserve_case = preserve_case
        self.use_unicode = use_unicode

    def tokenize(self, s):
        """
        Argument: s -- any string or unicode object
        Value: a tokenize list of strings; conatenating this list returns the original string if preserve_case=False
        """
        # Try to ensure unicode:
        if self.use_unicode:
            try:
                s = str(s)
            except UnicodeDecodeError:
                s = str(s).encode('string_escape')
                s = str(s)
        # Fix HTML character entitites:
        s = self.__html2unicode(s)
        s = self.__removeHex(s)
        # Tokenize:
        words = word_re.findall(s)
        #print words #debug
        # Possible alter the case, but avoid changing emoticons like :D into :d:
        if not self.preserve_case:
            words = map((lambda x : x if emoticon_re.search(x) else x.lower()), words)

        return words


    def __html2unicode(self, s):
        """
        Internal metod that seeks to replace all the HTML entities in
        s with their corresponding unicode characters.
        """
        # First the digits:
        ents = set(html_entity_digit_re.findall(s))
        if len(ents) > 0:
            for ent in ents:
                entnum = ent[2:-1]
                try:
                    entnum = int(entnum)
                    s = s.replace(ent, chr(entnum))
                except:
                    pass
        # Now the alpha versions:
        ents = set(html_entity_alpha_re.findall(s))
        ents = filter((lambda x : x != amp), ents)
        for ent in ents:
            entname = ent[1:-1]
            try:
                s = s.replace(ent, chr(htmlentitydefs.name2codepoint[entname]))
            except:
                pass
            s = s.replace(amp, " and ")
        return s

    def __removeHex(self, s):
        return hex_re.sub(' ', s)


In [ ]:
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from io import StringIO
def LeXmo(text,document,dictionary):

    '''
      Takes text and adds if to a dictionary with 10 Keys  for each of the 10 emotions in the NRC Emotion Lexicon,
      each dictionay contains the value of the text in that emotions divided to the text word count
      INPUT: string
      OUTPUT: dictionary with the text and the value of 10 emotions
      '''
    reponse = ""
    choice = 0
    df = pd.DataFrame()
    emodic = {'text': text}
    if(dictionary == "all"):
        #first nrc
        choice = 1
    if(dictionary == "nrc" or choice == 1):
        response = requests.get('https://raw.github.com/dinbav/LeXmo/master/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
        nrc = StringIO(response.text)


#        emodic = {'anger': [], 'anticipation': [], 'disgust': [], 'fear': [], 'joy': [], 'negative': [],                  'positive': [], 'sadness': [], 'surprise': [], 'trust': []}
        thisdic =  {'anger': [], 'anticipation': [], 'disgust': [], 'fear': [], 'joy': [], 'negative': [],'positive': [], 'sadness': [], 'surprise': [], 'trust': []}
        emodic = Merge(emodic,thisdic)

        lexicon = pd.read_csv(nrc,
                            names=["word", "emotion", "association"],
                            sep=r'\t', engine='python')
        df = df.append(lexicon)

    if(dictionary == "liwc" or choice == 1):
        #response = requests.get('/home/kokil/feature_extraction/data/liwc2015.txt')
        #liwc = StringIO(response.text)
        liwc = '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/liwc2015.txt'


       # emodic = {'text': text, 'PPRON': [],'BODY': [],'WE': [],'DEATH': [],'FOCUSFUTURE': [],'FEEL': [],'INTERROG': [],'NUMBER': [],'POSEMO': [],'NEGATE': [],'QUANT': [],'THEY': [],'AFFECT': [],'RELATIV': [],'HOME': [],'CONJ': [],'COGPROC': [],'SEXUAL': [],'AUXVERB': [],'SHEHE': [],'BIO': [],'DIFFER': [],'POWER': [],'NETSPEAK': [],'INFORMAL': [],'CAUSE': [],'FILLER': [],'INSIGHT': [],'LEISURE': [],'NEGEMO': [],'MOTION': [],'SEE': [],'FOCUSPAST': [],'ANGER': [],'ARTICLE': [],'NONFLU': [],'MALE': [],'WORK': [],'FRIEND': [],'FUNCTION': [],'RISK': [],'FAMILY': [],'SPACE': [],'I': [],'IPRON': [],'SOCIAL': [],'ASSENT': [],'DRIVES': [],'PERCEPT': [],'VERB': [],'HEAR': [],'FEMALE': [],'DISCREP': [],'YOU': [],'ADJ': [],'ACHIEVE': [],'RELIG': [],'TENTAT': [],'COMPARE': [],'ADVERB': [],'PRONOUN': [],'MONEY': [],'FOCUSPRESENT': [],'INGEST': [],'AFFILIATION': [],'SWEAR': [],'HEALTH': [],'SAD': [],'TIME': [],'REWARD': [],'ANX': [],'PREP': [],'CERTAIN': []}
        thisdic = {'PPRON': [],'BODY': [],'WE': [],'DEATH': [],'FOCUSFUTURE': [],'FEEL': [],'INTERROG': [],'NUMBER': [],'POSEMO': [],'NEGATE': [],'QUANT': [],'THEY': [],'AFFECT': [],'RELATIV': [],'HOME': [],'CONJ': [],'COGPROC': [],'SEXUAL': [],'AUXVERB': [],'SHEHE': [],'BIO': [],'DIFFER': [],'POWER': [],'NETSPEAK': [],'INFORMAL': [],'CAUSE': [],'FILLER': [],'INSIGHT': [],'LEISURE': [],'NEGEMO': [],'MOTION': [],'SEE': [],'FOCUSPAST': [],'ANGER': [],'ARTICLE': [],'NONFLU': [],'MALE': [],'WORK': [],'FRIEND': [],'FUNCTION': [],'RISK': [],'FAMILY': [],'SPACE': [],'I': [],'IPRON': [],'SOCIAL': [],'ASSENT': [],'DRIVES': [],'PERCEPT': [],'VERB': [],'HEAR': [],'FEMALE': [],'DISCREP': [],'YOU': [],'ADJ': [],'ACHIEVE': [],'RELIG': [],'TENTAT': [],'COMPARE': [],'ADVERB': [],'PRONOUN': [],'MONEY': [],'FOCUSPRESENT': [],'INGEST': [],'AFFILIATION': [],'SWEAR': [],'HEALTH': [],'SAD': [],'TIME': [],'REWARD': [],'ANX': [],'PREP': [],'CERTAIN': []}
        emodic = Merge(emodic,thisdic)
        lexicon = pd.read_csv(liwc,
                            names=["word", "emotion", "association"],
                            sep=r'\t', engine='python')
        df = df.append(lexicon)

    if(dictionary == "delib" or choice == 1):
        delib = '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/dd_delib.txt'
        #response = requests.get('/home/kokil/feature_extraction/data/dd_delib.txt')
        #delib = StringIO(response.text)



       # emodic = {'text': text, 'EMP_RES': [],'UNCIVIL_ABUSE': [],'CONSTRUCTIVENESS': [],'JUSTIFICATION': [],'RECIPROCITY': [],'JUST_EXT': [],'RELEVANCE': [],'JUST_INT': []}
        thisdic =  {'EMP_RES': [],'UNCIVIL_ABUSE': [],'CONSTRUCTIVENESS': [],'JUSTIFICATION': [],'RECIPROCITY': [],'JUST_EXT': [],'RELEVANCE': [],'JUST_INT': []}
        emodic = Merge(emodic,thisdic)


        lexicon = pd.read_csv(delib,
                            names=["word", "emotion", "association"],
                            sep=r'\t', engine='python')
        df = df.append(lexicon)
    if(dictionary == "hate" or choice == 1):
        hate = '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/incivilities.txt'
        #response = requests.get('/home/kokil/feature_extraction/data/incivilities.txt')
        #hate = StringIO(response.text)



        #emodic = {'text': text, 'SWEAR': [],'UNCIV': [],'OFFEN': []}
        thisdic = {'UNCIV': [],'OFFEN': []}
        emodic = Merge(emodic,thisdic)

        lexicon = pd.read_csv(hate,           names=["word", "emotion", "association"],             sep=r'\t', engine='python')
        df = df.append(lexicon)

    df = df.drop_duplicates(subset=['word', 'emotion'])
    df.reset_index()
    emolex_words = df.pivot(index='word',
                                   columns='emotion',
                                   values='association').reset_index()
    emolex_words.drop(emolex_words.index[0])

    categories = emolex_words.columns.drop('word')

    stemmer = SnowballStemmer("english")

    rows_list = []
    word_count = len(document)
    for word in document:
            word = stemmer.stem(word.lower())

            emo_score = (emolex_words[emolex_words.word == word])
            rows_list.append(emo_score)


    df = pd.concat(rows_list)
    df.reset_index(drop=True)

    for category in list(categories):
        emodic[category] = df[category].sum() / word_count

    return emodic

In [ ]:
import csv

tokenizer = Tokenizer()

def tokenize_messages(filename,col_text,col_msgid):
    with open(filename,encoding="utf-8") as corpus:
            reader = csv.reader(corpus)
    #splitsfile = open('C:/Users/User/Dropbox/Content Analysis/Corpus/Full Corpus/fullcorpus_split.csv','a',newline='',encoding="utf-8")
    #f_revs = csv.writer(splitsfile)
    #f_revs.writerow(["message_id","SITE ID","message","Like Count","postlength"])

            rows_list = []
            for row in reader:
                message = row[col_text]
                tokenizer = Tokenizer(preserve_case=True)
                words = tokenizer.tokenize(message.lower())
                #print(words)
                totalGrams=0
                freqs = dict()
                totalChars = 0
                gram = ''
                for n in range (1,4):
                    for i in range(0,(len(words) - n)+1):
                        totalGrams += 1
                        gram = ' '.join(words[i:i+n])
                        try:
                            freqs[gram] = 1
                        except:
                            print("error")
                freqs["message_id"]=row[col_msgid]
                rows_list.append(freqs)
            df = pd.DataFrame(rows_list)
            df= df.replace(np.nan, 0)
            print("Writing tokenized messages to csv...")
            timestr = time.strftime("%Y%m%d-%H%M")
            #print timestr
            df.to_csv("tokenized_messages_"+timestr+".csv")
            return df

############################

def emolize_messages(filename,col_text,col_msgid,choice):
    with open(filename,encoding="utf-8") as corpus:
            reader = csv.reader(corpus)
    #splitsfile = open('C:/Users/User/Dropbox/Content Analysis/Corpus/Full Corpus/fullcorpus_split.csv','a',newline='',encoding="utf-8")
    #f_revs = csv.writer(splitsfile)
    #f_revs.writerow(["message_id","SITE ID","message","Like Count","postlength"])

            rows_list = []
            for row in reader:
                message = row[col_text]
                tokenizer = Tokenizer(preserve_case=True)
                words = tokenizer.tokenize(message.lower())
                emodic = LeXmo(message.lower(),words,choice)
                print(emodic)
                rows_list.append(emodic)
                #print(emodic)
            df = pd.DataFrame(rows_list)
            df= df.replace(np.nan, 0)
            print("Writing emolized messages to csv...")
            timestr = time.strftime("%Y%m%d-%H%M")
            #print timestr
            df.to_csv(filename+"_"+choice+"_"+timestr+".csv")
            return df

############################
def extract_counts(df,X_col):
            vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, stop_words='english', strip_accents='unicode')
            corpus = list(df[X_col].str.lower())
            X = vectorizer.fit_transform(corpus)
            df = df.join(pd.DataFrame(X.toarray()).add_prefix('count_'))
            df.to_csv(os.path.join(MODIFIED_DATA, '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/modified_data/counts.csv'))
def extract_tfidf(df,X_col):
            vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer, stop_words='english', strip_accents='unicode',max_features = 10000)
            corpus = list(df[X_col].str.lower())
            X = vectorizer.fit_transform(corpus)
            df = df.join(pd.DataFrame(X.toarray()).add_prefix('tfidf_'))
            df.to_csv(os.path.join(MODIFIED_DATA, '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/modified_data/tfidf.csv'))
            return df
def extract_tfidf_and_pos(df,X_col):
            vectorizer = TfidfVectorizer(    tokenizer=spacy_tokenizer,            preprocessor=preprocess,            ngram_range=(1, 3),            stop_words='english', #We do better when we keep stopwords
                                         use_idf=True,smooth_idf=False,      norm=None, #Applies l2 norm smoothin
                                         decode_error='replace',            max_features=10000,                         )#    min_df=1,   max_df=0.501
            tweets = df[X_col]
            tfidf = vectorizer.fit_transform(tweets).toarray()
            print("came here")
            vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names_out())}
            idf_vals = vectorizer.idf_
            idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores
            #Get POS tags for tweets and save as a string
            print("came here")
            tweet_tags = []
            for t in tweets:
                tokenizer = Tokenizer(preserve_case=True)
                tokens = tokenizer.tokenize(preprocess(t))
                tags = nltk.pos_tag(tokens)
                tag_list = [x[1] for x in tags]
                #for i in range(0, len(tokens)):
                tag_str = " ".join(tag_list)
                tweet_tags.append(tag_str)
            pos_vectorizer = TfidfVectorizer(tokenizer=None,    lowercase=False,    preprocessor=None,    ngram_range=(1, 3),    stop_words=None, #We do better when we keep stopwords
                                             use_idf=False,    smooth_idf=False,    norm=None, #Applies l2 norm smoothing
                                             decode_error='replace',    max_features=5000      )#  min_df=1,   max_df=0.501,
             #Construct POS TF matrix and get vocab dict
            pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
            pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names_out())}
            M = np.concatenate([tfidf,pos],axis=1)
            #Finally get a list of variable names
            variables = ['']*len(vocab)
            for k,v in vocab.items():
                variables[v] = 'tfidf_'+k
            pos_variables = ['']*len(pos_vocab)
            for k,v in pos_vocab.items():
                pos_variables[v] = 'POS_' + k

            feature_names = variables+pos_variables
            M_df = pd.DataFrame(M,columns = feature_names)
            df = df.join(M_df)


            ## save the pickles
            new_vocab = {v:i for i, v in enumerate(variables)}
            new_vectorizer = TfidfVectorizer(    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
                tokenizer=spacy_tokenizer,    preprocessor=preprocess,    ngram_range=(1, 3),    stop_words='english', #We do better when we keep stopwords
                use_idf=False,    smooth_idf=False,    norm=None, #Applies l2 norm smoothing
                decode_error='replace',    min_df=1,    max_df=1.0,    vocabulary=new_vocab    )
            joblib.dump(new_vectorizer, '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/final_tfidf_vectorizer.pkl')
            #Results are the same if use IDF but the problem is that IDF will be different if we use different data. Instead we have to use the original IDF scores and multiply them by the new matrix.
            joblib.dump(idf_vals, '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/final_idf_vectorizer.pkl')

            #This is simpler as we do not need to worry about IDF but it will be slower as we have to compute the POS tags for the new data. Here we can simply use the old POS tags.
            new_pos = {v:i for i, v in enumerate(pos_variables)}
            new_pos_vectorizer = TfidfVectorizer(    tokenizer=None,    lowercase=False,    preprocessor=None,    ngram_range=(1, 3),    stop_words=None, #We do better when we keep stopwords
                                                   use_idf=False,    smooth_idf=False,    norm=None, #Applies l2 norm smoothing
                                                 decode_error='replace',    min_df=1,    max_df=1.0,    vocabulary=new_pos    )
            joblib.dump(new_pos_vectorizer, '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/lexica/final_pos_vectorizer.pkl')
            return df
        #print(tokens[i],tag_list[i])

In [ ]:

# code
# Python code to merge dict using a single
# expression
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

def extract_feats(filename, X_col):
    df = emolize_messages( filename,1,0,"all")
    extract_harbingers(df, X_col)
    extract_politeness_feats(df, X_col)
    df = extract_tfidf_and_pos(df,X_col)
    timestr = time.strftime("%Y%m%d-%H%M")
    #print timestr
    df.to_csv(filename+"_allfeats_"+timestr+".csv")

In [ ]:
filename = "/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/data/sample.csv"
DATA_PATH = "/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/data/sample.csv"
OUTPUT_DIR = '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/data'     # You'll get 2 directories here, one will have t
X_col = 'text'  # Name of X column (string)
y_col = 'label'        # Name of y column (0/1)
MODIFIED_DATA = os.path.join(OUTPUT_DIR, '/content/drive/MyDrive/Colab_Notebooks/deliberative-politics-main/modified_data')
os.makedirs(MODIFIED_DATA, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
extract_feats(filename,"text")
##        "C:/Users/User/Dropbox/data/msgs_jun23.csv" --predict 1 0

<ipython-input-8-3e7099eac5d5>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)


{'text': 'text', 'anger': 0.0, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.0, 'joy': 0.0, 'negative': 0.0, 'positive': 0.0, 'sadness': 0.0, 'surprise': 0.0, 'trust': 0.0, 'PPRON': 0.0, 'BODY': 0.0, 'WE': 0.0, 'DEATH': 0.0, 'FOCUSFUTURE': 0.0, 'FEEL': 0.0, 'INTERROG': 0.0, 'NUMBER': 0.0, 'POSEMO': 0.0, 'NEGATE': 0.0, 'QUANT': 0.0, 'THEY': 0.0, 'AFFECT': 0.0, 'RELATIV': 0.0, 'HOME': 0.0, 'CONJ': 0.0, 'COGPROC': 0.0, 'SEXUAL': 0.0, 'AUXVERB': 0.0, 'SHEHE': 0.0, 'BIO': 0.0, 'DIFFER': 0.0, 'POWER': 0.0, 'NETSPEAK': 0.0, 'INFORMAL': 0.0, 'CAUSE': 0.0, 'FILLER': 0.0, 'INSIGHT': 0.0, 'LEISURE': 0.0, 'NEGEMO': 0.0, 'MOTION': 0.0, 'SEE': 0.0, 'FOCUSPAST': 0.0, 'ANGER': 0.0, 'ARTICLE': 0.0, 'NONFLU': 0.0, 'MALE': 0.0, 'WORK': 0.0, 'FRIEND': 0.0, 'FUNCTION': 0.0, 'RISK': 0.0, 'FAMILY': 0.0, 'SPACE': 0.0, 'I': 0.0, 'IPRON': 0.0, 'SOCIAL': 0.0, 'ASSENT': 0.0, 'DRIVES': 0.0, 'PERCEPT': 0.0, 'VERB': 0.0, 'HEAR': 0.0, 'FEMALE': 0.0, 'DISCREP': 0.0, 'YOU': 0.0, 'ADJ': 0.0, 'ACHIEVE': 0.0, 'RELIG': 0.

<ipython-input-8-3e7099eac5d5>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)


{'text': 'i think that just having the presence of a good guy with a gun would prevent any action by a bad guy with a gun.', 'anger': 0.11538461538461539, 'anticipation': 0.038461538461538464, 'disgust': 0.038461538461538464, 'fear': 0.15384615384615385, 'joy': 0.038461538461538464, 'negative': 0.11538461538461539, 'positive': 0.07692307692307693, 'sadness': 0.038461538461538464, 'surprise': 0.038461538461538464, 'trust': 0.038461538461538464, 'PPRON': 0.038461538461538464, 'BODY': 0.0, 'WE': 0.0, 'DEATH': 0.0, 'FOCUSFUTURE': 0.0, 'FEEL': 0.0, 'INTERROG': 0.0, 'NUMBER': 0.0, 'POSEMO': 0.038461538461538464, 'NEGATE': 0.0, 'QUANT': 0.0, 'THEY': 0.0, 'AFFECT': 0.07692307692307693, 'RELATIV': 0.0, 'HOME': 0.0, 'CONJ': 0.0, 'COGPROC': 0.07692307692307693, 'SEXUAL': 0.0, 'AUXVERB': 0.07692307692307693, 'SHEHE': 0.0, 'BIO': 0.0, 'DIFFER': 0.0, 'POWER': 0.0, 'NETSPEAK': 0.0, 'INFORMAL': 0.0, 'CAUSE': 0.0, 'FILLER': 0.0, 'INSIGHT': 0.038461538461538464, 'LEISURE': 0.0, 'NEGEMO': 0.0384615384615

<ipython-input-8-3e7099eac5d5>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)


...

{'text': 'having a gun does not lead to violence and gang, having a mentality that seeks to force your will on another does lead to gang and crime.', 'anger': 0.13793103448275862, 'anticipation': 0.034482758620689655, 'disgust': 0.0, 'fear': 0.10344827586206896, 'joy': 0.0, 'negative': 0.13793103448275862, 'positive': 0.06896551724137931, 'sadness': 0.0, 'surprise': 0.0, 'trust': 0.0, 'PPRON': 0.034482758620689655, 'BODY': 0.0, 'WE': 0.0, 'DEATH': 0.0, 'FOCUSFUTURE': 0.034482758620689655, 'FEEL': 0.0, 'INTERROG': 0.0, 'NUMBER': 0.0, 'POSEMO': 0.0, 'NEGATE': 0.034482758620689655, 'QUANT': 0.0, 'THEY': 0.0, 'AFFECT': 0.0, 'RELATIV': 0.10344827586206896, 'HOME': 0.0, 'CONJ': 0.06896551724137931, 'COGPROC': 0.10344827586206896, 'SEXUAL': 0.0, 'AUXVERB': 0.10344827586206896, 'SHEHE': 0.0, 'BIO': 0.0, 'DIFFER': 0.034482758620689655, 'POWER': 0.06896551724137931, 'NETSPEAK': 0.0, 'INFORMAL': 0.0, 'CAUSE': 0.06896551724137931, 'FILLER': 0.0, 'INSIGHT': 0.0, 'LEISURE': 0.0, 'NEGEMO': 0.0, 'MOTI

<ipython-input-8-3e7099eac5d5>:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)
<ipython-input-8-3e7099eac5d5>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lexicon)


{'text': 'all of these dangerous cases exist, only because guns exist in the first place. remove them all and violence will drop dramatically.', 'anger': 0.04, 'anticipation': 0.0, 'disgust': 0.0, 'fear': 0.12, 'joy': 0.0, 'negative': 0.12, 'positive': 0.0, 'sadness': 0.08, 'surprise': 0.0, 'trust': 0.0, 'PPRON': 0.04, 'BODY': 0.0, 'WE': 0.0, 'DEATH': 0.0, 'FOCUSFUTURE': 0.04, 'FEEL': 0.0, 'INTERROG': 0.0, 'NUMBER': 0.04, 'POSEMO': 0.0, 'NEGATE': 0.0, 'QUANT': 0.08, 'THEY': 0.04, 'AFFECT': 0.04, 'RELATIV': 0.16, 'HOME': 0.0, 'CONJ': 0.04, 'COGPROC': 0.08, 'SEXUAL': 0.0, 'AUXVERB': 0.04, 'SHEHE': 0.0, 'BIO': 0.0, 'DIFFER': 0.0, 'POWER': 0.0, 'NETSPEAK': 0.0, 'INFORMAL': 0.0, 'CAUSE': 0.0, 'FILLER': 0.0, 'INSIGHT': 0.0, 'LEISURE': 0.0, 'NEGEMO': 0.04, 'MOTION': 0.04, 'SEE': 0.0, 'FOCUSPAST': 0.0, 'ANGER': 0.0, 'ARTICLE': 0.04, 'NONFLU': 0.0, 'MALE': 0.0, 'WORK': 0.0, 'FRIEND': 0.0, 'FUNCTION': 0.28, 'RISK': 0.04, 'FAMILY': 0.0, 'SPACE': 0.08, 'I': 0.0, 'IPRON': 0.04, 'SOCIAL': 0.04, 'ASS

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ca', 'nt'] not in stop_words.
  warnings.warn(


came here
came here
